# AOC 2022

In [ ]:
\cd notebooks

'io



# Advent of Code 2022 - 04

URL: https://adventofcode.com/2022/day/4

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/04.k):

In [28]:
`0:$(+/1>*/-/)'1{|y}\\`I$+'"-"\''+","\'0:"i/04" /both parts

503
827


In [34]:
 /`0:$(+/1>*/-/)'1{|y}\\`I$+'"-"\''+
`0:$(/-/)'1{|y}\\`I$+'"-"\''+","\'0:"i/04" /both parts

(-1 0 24 -1 1 0 -2 0 0 1 0 -27 -41 0 -25 1 13 0 -8 -6 -6 76 -70 83 -1 -1 0 1 1 -1 38 83 -21 0 1 9 0 -8 0 26 -1 -2 -1 5 -1 -9 -1 65 22 -14 0 70 15 -1 -1 84 48 39 63 -16 0 1 0 28 40 -3 -1 0 1 -30 0 -1 1 33 -1 -2 -6 -1 -7 -1 -15 -45 -2 0 1 -1 80 -1 1 -1 -1 -1 1 22 0 35 53 1 7 0 1 2 -4 0 -1 -12 0 72 -66 -1 37 -1 0 49 -1 -76 -1 36 -56 -5 0 23 96 -1 87 -1 -52 -26 0 32 1 -55 0 -91 3 -1 83 -7 -1 1 0 47 -57 0 -1 -1 -9 0 20 9 -27 0 61 -1 48 -72 -1 -15 -1 -9 85 0 -1 95 -4 14 -1 -35 38 0 1 3 -1 -36 -40 1 -28 93 -1 -21 0 11 19 -1 -31 -1 -1 69 0 76 -3 55 -1 1 -15 77 1 -1 -1 6 0 0 1 -4 -2 -19 -5 -1 -51 0 1 1 0 -1 -1 -1 -1 -39 68 -9 -62 1 43 -3 -91 -3 -1 0 1 13 -66 1 -62 -37 1 -31 58 -1 -29 83 0 -1 26 40 -1 -8 -1 1 1 33 0 22 0 1 -38 0 1 -9 0 1 -2 1 1 0 -1 -81 -1 7 0 -17 0 -23 37 1 -79 94 -86 -1 0 1 -2 -13 0 -43 -17 6 11 -1 0 -65 -1 -1 -11 -91 51 -1 1 -1 -45 0 -1 1 -1 -28 0 -94 -87 -1 1 19 0 -19 0 -32 1 2 -19 -1 -27 0 6 0 -1 47 8 -14 0 0 0 -89 -78 25 35 0 1 0 -1 24 1 0 2 -36 -3 -40 73 -1 -10 -58 -1 3 1